In [37]:
import glob
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB



In [38]:
def read_code(lang):
    files = glob.glob('benchmarks/benchmarksgame/bench/fasta/*.{}'.format(lang))
    texts = []
    for file in files:
        with open(file) as f:
            texts.append(f.read())
    print(texts)
    return texts

In [39]:
clojure = glob.glob('benchmarks/benchmarksgame/bench/fasta/*.clojure')

In [44]:
def read_code(lang):    
    clojure = glob.glob('benchmarks/benchmarksgame/bench/fasta/*.{}'.format(lang))
    text = []
    for file in clojure:
        with open(file) as f:
            text.append(f.read())
    print(text)

In [50]:
clojure = glob.glob('benchmarks/benchmarksgame/bench/fasta/*.clojure')
text = []
for file in clojure:
    with open(file) as f:
        text.append(f.read())
print(text)

[';;   The Computer Language Benchmarks Game\n;;   http://benchmarksgame.alioth.debian.org/\n\n;; contributed by Andy Fingerhut\n\n(ns fasta\n  (:gen-class))\n\n(set! *warn-on-reflection* true)\n\n;; Handle slight difference in function name between Clojure 1.2.0 and\n;; 1.3.0-alpha*.\n(defmacro my-unchecked-inc-int [& args]\n  (if (and (== (*clojure-version* :major) 1)\n           (== (*clojure-version* :minor) 2))\n    `(unchecked-inc ~@args)\n    `(unchecked-inc-int ~@args)))\n\n(defmacro my-unchecked-add-int [& args]\n  (if (and (== (*clojure-version* :major) 1)\n           (== (*clojure-version* :minor) 2))\n    `(unchecked-add ~@args)\n    `(unchecked-add-int ~@args)))\n\n(defmacro my-unchecked-multiply-int [& args]\n  (if (and (== (*clojure-version* :major) 1)\n           (== (*clojure-version* :minor) 2))\n    `(unchecked-multiply ~@args)\n    `(unchecked-multiply-int ~@args)))\n\n(defmacro my-unchecked-remainder-int [& args]\n  (if (and (== (*clojure-version* :major) 1)\n     

In [46]:
print(cloj)

None
